In [ ]:
import torch
print(torch.cuda.is_available())


True


In [ ]:
!pip install -U transformers datasets accelerate peft trl bitsandbytes wandb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
pip install trl==0.12.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.15.2
    Uninstalling trl-0.15.2:
      Successfully uninstalled trl-0.15.2


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model
from datasets import load_dataset, DatasetDict
from trl import SFTTrainer, SFTConfig
import torch, wandb

In [ ]:
# Set your model repository ID (ensure you have access)
base_model = "meta-llama/Llama-3.2-1B"

# Configure BitsAndBytes for 4-bit loading (saves VRAM on T4)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",                # nf4 for efficient quantization
    bnb_4bit_compute_dtype=torch.float16,     # use float16 compute for speed
    bnb_4bit_use_double_quant=True
)

# Load model from the Hub
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation="eager"
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [ ]:
# Define the PEFT/LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model = get_peft_model(model, peft_config)



In [ ]:
dataset = load_dataset("brainchalov/pubmed_arxiv_abstracts_data")

# Define a formatting function for PubMed/ArXiv abstracts
def format_pubmed_example(example):
    # Create a prompt using title, abstract, journal, and field information
    prompt = (
        f"Title: {example['title']}\n"
        f"Abstract: {example['abstr']}\n"
        f"Journal: {example['journal']}\n"
        f"Field: {example['field']}\n"
        "Summary:"
    )
    example["text"] = prompt
    return example

# Apply the formatting function to the entire dataset
dataset = dataset.map(format_pubmed_example, num_proc=4)


README.md:   0%|          | 0.00/391 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


df_0.csv.gz:   0%|          | 0.00/25.3M [00:00<?, ?B/s]

df_1.csv.gz:   0%|          | 0.00/25.4M [00:00<?, ?B/s]

df_2.csv.gz:   0%|          | 0.00/25.4M [00:00<?, ?B/s]

...

df_7.csv.gz:   0%|          | 0.00/25.5M [00:00<?, ?B/s]

df_8.csv.gz:   0%|          | 0.00/25.5M [00:00<?, ?B/s]

df_9.csv.gz:   0%|          | 0.00/28.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/500335 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/500335 [00:00<?, ? examples/s]

In [ ]:
# Step 1: Do a train_test_split on the 'train' dataset
split_dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)

# Step 2: Wrap it in a new DatasetDict with 'train' and 'validation'
dataset = DatasetDict({
    "train": split_dataset["train"],
    "validation": split_dataset["test"]
})

# Define the SFT configuration (replacing TrainingArguments)
sft_config = SFTConfig(
    output_dir="llama3-1-3b-pubmed-finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,  # Adjust as needed
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    learning_rate=2e-4,
    fp16=True,
    group_by_length=True,
    max_seq_length=512,
    dataset_text_field="text",
    packing=False
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['abstr', 'title', 'journal', 'field', 'label_journal', 'label_field', 'text'],
        num_rows: 500335
    })
})


In [ ]:
# Define the SFT configuration
sft_config = SFTConfig(
    output_dir="llama3-1-3b-pubmed-finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,  # Adjust as needed
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    learning_rate=2e-4,
    fp16=True,
    group_by_length=True,
    max_seq_length=512,
    dataset_text_field="text",
    packing=False
)

# Set a padding token (if not already set) and resize model embeddings accordingly
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))

# Initialize the trainer with the SFTConfig
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    peft_config=peft_config,
    tokenizer=tokenizer,
    args=sft_config
)

# Start training
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/450301 [00:00<?, ? examples/s]

Map:   0%|          | 0/50034 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ratan152004 (ratan152004-dayananda-sagar-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetunin

In [ ]:

# Example inference: generate a sample summary
sample_prompt = (
    "Title: Example Study on Neural Networks\n"
    "Abstract: This study explores the applications of deep learning in healthcare, "
    "demonstrating improved diagnostic accuracy.\n"
    "Journal: Journal of AI Research\n"
    "Field: Healthcare\n"
    "Summary:"
)
inputs = tokenizer(sample_prompt, return_tensors='pt', padding=True, truncation=True, max_length=512).to("cuda")
outputs = model.generate(**inputs, max_length=150, num_return_sequences=1)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# Save the fine-tuned model
trainer.model.save_pretrained("llama3-1-3b-pubmed-finetuned")